<a href="https://colab.research.google.com/github/shelly-hub/Home_Sales/blob/main/Home_Sales_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [833 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [979 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:13 https://developer.download.nvidia.com

In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header = True, inferSchema = True, timestampFormat = "yyyy/MM/dd" )

df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [ ]:
# 2. Create a temporary view of the DataFrame.

df.createOrReplaceTempView('home')

In [ ]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

spark.sql(""" SELECT YEAR(date) AS year, bedrooms, ROUND(AVG(price), 2) AS average_price
              FROM home
              WHERE bedrooms = "4"
              GROUP BY 1,2
              ORDER BY 1""").show()

+----+--------+-------------+
|year|bedrooms|average_price|
+----+--------+-------------+
|2019|       4|     300263.7|
|2020|       4|    298353.78|
|2021|       4|    301819.44|
|2022|       4|    296363.88|
+----+--------+-------------+



In [ ]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

spark.sql(""" SELECT YEAR(date) AS year, bedrooms, bathrooms, ROUND(AVG(price), 2) AS average_price
              FROM home
              WHERE bedrooms = "3" AND bathrooms = "3"
              GROUP BY 1,2,3
              ORDER BY 1""").show()


+----+--------+---------+-------------+
|year|bedrooms|bathrooms|average_price|
+----+--------+---------+-------------+
|2019|       3|        3|    287287.82|
|2020|       3|        3|    294204.16|
|2021|       3|        3|    294211.46|
|2022|       3|        3|    292725.69|
+----+--------+---------+-------------+



In [ ]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

spark.sql(""" SELECT YEAR(date) AS year, bedrooms, bathrooms, floors,
              ROUND(AVG(price) FILTER(WHERE sqft_living >= 2000), 2) AS AVG_PRICE_OVER_2000sqft
              FROM home
              WHERE bedrooms = "3" AND bathrooms = "3" AND floors = "2"
              GROUP BY 1,2, 3, 4
              ORDER BY 1""").show()

+----+--------+---------+------+-----------------------+
|year|bedrooms|bathrooms|floors|AVG_PRICE_OVER_2000sqft|
+----+--------+---------+------+-----------------------+
|2019|       3|        3|     2|              289859.14|
|2020|       3|        3|     2|              292289.09|
|2021|       3|        3|     2|              296330.96|
|2022|       3|        3|     2|              290242.99|
+----+--------+---------+------+-----------------------+



In [ ]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql( """ SELECT view,  CAST(AVG(price) as decimal (10,2)) AS average_price_above350k FROM home
            GROUP BY  1
            HAVING AVG(price) > 350000
            ORDER BY 2
            """).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-----------------------+
|view|average_price_above350k|
+----+-----------------------+
|  70|              695865.58|
|  63|              711614.55|
|  66|              712475.00|
|  68|              716785.44|
|  56|              718176.40|
|  52|              733780.26|
|  57|              734340.50|
|  65|              736679.93|
|  67|              737970.96|
|  74|              745077.00|
|  61|              746877.59|
|  69|              750537.94|
|  73|              752861.18|
|  60|              754939.65|
|  53|              755214.80|
|  62|              759150.14|
|  58|              759764.65|
|  64|              767036.67|
|  55|              771153.32|
|  71|              775651.10|
+----+-----------------------+
only showing top 20 rows

--- 0.6846990585327148 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home")

DataFrame[]

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home')

True

In [ ]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql( """ SELECT view,  CAST(AVG(price) as decimal (10,2)) AS average_price_above350k FROM home
            GROUP BY  1
            HAVING AVG(price) > 350000
            ORDER BY 2
            """).show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+-----------------------+
|view|average_price_above350k|
+----+-----------------------+
|  70|              695865.58|
|  63|              711614.55|
|  66|              712475.00|
|  68|              716785.44|
|  56|              718176.40|
|  52|              733780.26|
|  57|              734340.50|
|  65|              736679.93|
|  67|              737970.96|
|  74|              745077.00|
|  61|              746877.59|
|  69|              750537.94|
|  73|              752861.18|
|  60|              754939.65|
|  53|              755214.80|
|  62|              759150.14|
|  58|              759764.65|
|  64|              767036.67|
|  55|              771153.32|
|  71|              775651.10|
+----+-----------------------+
only showing top 20 rows

--- 0.34937191009521484 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy('date_built').mode('overwrite').parquet('date_partitioned')

In [ ]:
# 11. Read the parquet formatted data.
p_df = spark.read.parquet('date_partitioned')

In [ ]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('p_home')

In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql( """ SELECT view,  CAST(AVG(price) as decimal (10,2)) AS average_price_above350k FROM p_home
            GROUP BY  1
            HAVING AVG(price) > 350000
            ORDER BY 2
            """).show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+-----------------------+
|view|average_price_above350k|
+----+-----------------------+
|  70|              695865.58|
|  63|              711614.55|
|  66|              712475.00|
|  68|              716785.44|
|  56|              718176.40|
|  52|              733780.26|
|  57|              734340.50|
|  65|              736679.93|
|  67|              737970.96|
|  74|              745077.00|
|  61|              746877.59|
|  69|              750537.94|
|  73|              752861.18|
|  60|              754939.65|
|  53|              755214.80|
|  62|              759150.14|
|  58|              759764.65|
|  64|              767036.67|
|  55|              771153.32|
|  71|              775651.10|
+----+-----------------------+
only showing top 20 rows

--- 0.9903292655944824 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home")

DataFrame[]

In [ ]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home')


False